hw2

In [26]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch

def save_result(preds):
    with open('test_result.csv', 'w', encoding='utf-8') as f:
        f.write('Id,Class\n')
        for i, pred in enumerate(preds):
            f.write('%d,%d\n' % (i, pred))
    
    return

data_root = './timit_11/'

train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of train data: ', train.shape)
print('Size of train label: ', train_label.shape)
print('Size of test data: ', test.shape)

class TIMITDataset(Dataset): 
    def __init__(self, data, label, mode="train"):
        self.mode = mode
        self.data = pd.DataFrame(data)
        self.label = pd.DataFrame(label)

        # 需要区分 train data 和 test data
        if self.mode == "train":
            indices = [i for i in range(len(self.data)) if i % 10 != 0]
            self.data = self.data.iloc[indices, :]
            self.label = self.label.iloc[indices]

        elif self.mode == "dev":
            indices = [i for i in range(len(self.data)) if i % 10 == 0]
            self.data = self.data.iloc[indices, :]
            self.label = self.label.iloc[indices]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.mode == "train" or self.mode == "dev":
            X = torch.tensor(self.data.iloc[idx].values.reshape(11, 39)[3:8, :].flatten(), dtype=torch.float32)
            y = torch.tensor(int(self.label.iloc[idx]), dtype=torch.int32)
            return X, y
        elif self.mode == "test":
            return torch.tensor(self.data.iloc[idx].values.reshape(11, 39)[3:8, :].flatten(), dtype=torch.float32)
        
train_data = TIMITDataset(train, train_label, mode="train")
dev_data = TIMITDataset(train, train_label, mode="dev")
test_data = TIMITDataset(test, None, mode="test")

train_dataloader = DataLoader(train_data, batch_size=100, shuffle=True)
dev_dataloader = DataLoader(dev_data, batch_size=100, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=100, shuffle=False)


Size of train data:  (1229932, 429)
Size of train label:  (1229932,)
Size of test data:  (451552, 429)


In [18]:
print(train_data.__len__())
print(train_data.data[0])
# 判断变量类型
# print(train[0])
# print(type(train[0]))

# 判断变量类型
print(test[0])
# print(type(test[0]))


# print(train_label[0])
# print(type(train_label[0]))
# print(type(train_data))
# print(type(train_data[0]))


1106938
1         -1.060318
2         -1.061233
3         -1.062128
4         -1.061916
5         -1.050420
             ...   
1229926    0.177523
1229927    0.138306
1229928    0.065482
1229929    0.057637
1229931    0.111426
Name: 0, Length: 1106938, dtype: float64
[-8.15812826e-01 -3.48689795e-01  3.95477504e-01 -1.63738783e-02
  8.60317469e-01  1.44315893e-02  3.84072900e-01  9.11684811e-01
  2.74616122e-01  7.21527755e-01  1.62349343e+00  1.94353950e+00
  9.64264333e-01 -7.72595452e-03 -1.81192949e-01  5.74482083e-01
 -2.26134375e-01  4.03959244e-01  3.92566890e-01  4.68575805e-01
  9.03372943e-01  9.12166536e-02  6.29455924e-01  6.50277019e-01
 -7.98417151e-01 -5.07435977e-01 -1.93714548e-03 -4.96793658e-01
 -1.30517155e-01  1.90668270e-01  3.52899522e-01  1.43388236e+00
  5.82235932e-01 -4.42176044e-01  3.92167211e-01  3.82801175e-01
 -5.74597836e-01 -9.66160059e-01  8.36981013e-02 -8.17853749e-01
 -3.47191900e-01  5.85275650e-01 -1.76819056e-01  6.26596749e-01
 -2.08324268e-01

In [27]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5*39, 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 39)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.05)

loss_record = {"train": [], "dev": []}
pred_record = []
target_record = []

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_record["train"].append(loss.item())

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def dev(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return test_loss



Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=195, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.2, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.2, inplace=False)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Linear(in_features=256, out_features=128, bias=True)
    (12): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): Linear(in_features=128, out_features=39, bias=True)
  )
)


In [28]:
epochs = 20
min_loss = 1.0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    dev_loss = dev(dev_dataloader, model, loss_fn)

    if dev_loss < min_loss:
        min_loss = dev_loss
        torch.save(model.state_dict(), "model.pth")
print("Done!")

Epoch 1
-------------------------------
loss: 3.718244  [  100/1106938]


/var/folders/2c/pn5qg61n16nfl0b9y726w55r0000gn/T/ipykernel_12492/1375298387.py:48: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  y = torch.tensor(int(self.label.iloc[idx]), dtype=torch.int32)


loss: 1.924306  [10100/1106938]
loss: 1.629065  [20100/1106938]
loss: 1.581387  [30100/1106938]
loss: 1.278219  [40100/1106938]
loss: 1.398767  [50100/1106938]
loss: 1.523835  [60100/1106938]
loss: 1.592161  [70100/1106938]
loss: 1.461328  [80100/1106938]
loss: 1.513250  [90100/1106938]
loss: 1.552409  [100100/1106938]
loss: 1.358105  [110100/1106938]
loss: 1.402215  [120100/1106938]
loss: 1.210059  [130100/1106938]
loss: 1.369584  [140100/1106938]
loss: 1.331123  [150100/1106938]
loss: 1.217478  [160100/1106938]
loss: 1.246017  [170100/1106938]
loss: 1.187148  [180100/1106938]
loss: 1.485128  [190100/1106938]
loss: 1.252234  [200100/1106938]
loss: 1.257684  [210100/1106938]
loss: 1.434850  [220100/1106938]
loss: 1.208052  [230100/1106938]
loss: 1.276450  [240100/1106938]
loss: 1.359486  [250100/1106938]
loss: 1.121016  [260100/1106938]
loss: 1.124274  [270100/1106938]
loss: 1.174506  [280100/1106938]
loss: 1.395367  [290100/1106938]
loss: 1.599495  [300100/1106938]
loss: 1.424653  [31

In [29]:
def test(dataloader, model):
    del model
    model = NeuralNetwork()
    ckpt = torch.load("model.pth")
    model.load_state_dict(ckpt)
    model.eval()

    preds = []
    with torch.no_grad():
        for X in dataloader:
            pred = model(X)
            pred = pred.squeeze()
            preds.append(pred.argmax(1))
    
    preds = torch.cat(preds, dim=0).long()
    print(preds)
    save_result(preds)
    
test(test_dataloader, model)

tensor([36, 36, 36,  ..., 30, 30, 25])
